In [1]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
import os
from osgeo import gdal
from scipy import stats
import math
from math import log10, sqrt
import pysptools.eea as eea
import pysptools.abundance_maps as amp
from scipy.optimize import nnls
from sklearn.preprocessing import normalize
from cvxopt import solvers, matrix

In [2]:
def qunan(img):
    gray = img[~np.isnan(img)]
    return gray
def baocun(gray,name):
    im_width = dataset.RasterXSize    #栅格矩阵的列数
    im_height = dataset.RasterYSize   #栅格矩阵的行数
    im_geotrans = dataset.GetGeoTransform()  #仿射矩阵
    im_proj = dataset.GetProjection() #地图投影信息
    driver = gdal.GetDriverByName("GTiff")                
    dataset2 = driver.Create(name, im_width, im_height, 1, gdal.GDT_Float32,options=['COMPRESS=LZW'])
    dataset2.SetGeoTransform(im_geotrans)              #写入仿射变换参数
    dataset2.SetProjection(im_proj)
    dataset2.GetRasterBand(1).WriteArray(gray)
    del dataset2
def baocun2(band_list,savepath):
    im_width = dataset.RasterXSize    #栅格矩阵的列数
    im_height = dataset.RasterYSize   #栅格矩阵的行数
    im_geotrans = dataset.GetGeoTransform()  #仿射矩阵
    im_proj = dataset.GetProjection() #地图投影信息
    driver = gdal.GetDriverByName("GTiff")            
    dataset2 = driver.Create(savepath, im_width, im_height, band_list.shape[0], gdal.GDT_Float32,options=['COMPRESS=LZW'])
    dataset2.SetGeoTransform(im_geotrans)              #写入仿射变换参数
    dataset2.SetProjection(im_proj)
    for v in range(band_list.shape[0]):
        dataset2.GetRasterBand(v+1).WriteArray(band_list[v])
    del dataset2

In [3]:
filename_list_all = filedialog.askopenfilenames()
%gui tk

In [4]:
filename_list_all

('D:/xin/无人机覆盖度模型/一种新的SAV提取方法/sav0091_transparent_mosaic_group1.tif',)

In [16]:
#path = 'D:/xin/无人机覆盖度模型/一种新的SAV提取方法/76-80_transparent_mosaic_group1.tif'
path = 'D:/xin/无人机覆盖度模型/一种新的SAV提取方法/9091_transparent_mosaic_group1.tif'
#path = 'D:/xin/无人机覆盖度模型/裁剪/bifenggang/bifenggang_481.tif'
#path = 'D:/xin/无人机覆盖度模型/bifenggang.tif'
dataset = gdal.Open(path)
#data = dataset.ReadAsArray()
b=dataset.GetRasterBand(1).ReadAsArray()
g=dataset.GetRasterBand(2).ReadAsArray()
r=dataset.GetRasterBand(3).ReadAsArray()
m,n = b.shape
RGB_img = cv2.merge([b,g,r])
HSV_img = cv2.cvtColor(RGB_img ,cv2.COLOR_BGR2HSV)
h = HSV_img[:,:,0:1].reshape(m,n)
s = HSV_img[:,:,1:2].reshape(m,n)
v = HSV_img[:,:,2:3].reshape(m,n)
#hsv_sample = np.stack((h, s, v), axis=-1).reshape((-1, 3))
#rgb_sample = np.stack((r, g, b), axis=-1).reshape((-1, 3))
#h1 = h[(h>0)&(s>0)&(v>0)].reshape(-1,1)
#s1 = s[(h>0)&(s>0)&(v>0)].reshape(-1,1)
#v1 = v[(h>0)&(s>0)&(v>0)].reshape(-1,1)
#hsv_sample_nozero = np.dstack((h1, s1, v1)).reshape(-1,3)
#
#r1 = r[(r>0)&(r>0)&(r>0)].reshape(-1,1)
#g1 = g[(g>0)&(g>0)&(g>0)].reshape(-1,1)
#b1 = b[(b>0)&(b>0)&(b>0)].reshape(-1,1)
#rgb_sample_nozero = np.dstack((r1, b1, g1)).reshape(-1,3)

In [18]:
final = np.copy(v)

In [19]:
final = np.where(final == 0, np.nan, final)
final[(final>=159)]=0
final[(final>1)&(final<159)]=1

In [20]:
#f = 2*s-h-v
baocun(final,'D:/xin/无人机覆盖度模型/一种新的SAV提取方法/9091_transparent_mosaic_group1(hsv_v).tif')

In [17]:
final = np.array([h,s,v])
baocun2(final,'D:/xin/无人机覆盖度模型/一种新的SAV提取方法/9091_transparent_mosaic_group1(hsv).tif')

In [ ]:
#注：1. ndindr不需要（m,n,p）格式的M，（m*n，1，p）结果一样
#2. normalized_matrix = matrix / np.sum(matrix, axis=1, keepdims=True)

In [32]:
###### RGB ############
#纯净像元目视结果
#高生长型（80，144，144）
#草甸（84，154，141）
#水体（97，175，148）
#纯净像元ndindr（未去0）
#[173. 215. 173.]
# [ 95. 180. 143.]
# [ 93. 136. 145.]
#纯净像元ndindr（去0）
#[[159. 100. 188.]
# [ 98.  69.  92.]
# [179. 176. 217.]]

#########HSV#########
#纯净像元目视结果
#高生长型[29,81,116]
#草甸  [35,114,154]
#水体  [40,112,175]
#纯净像元ndindr（去0）
#[[ 27. 140. 102.]
# [ 30.  46. 110.]
# [ 40.  77. 249.]]


In [89]:
#NFINDR-HSV（no-0）-3
nfindr = eea.NFINDR()
extract = nfindr.extract(hsv_sample_nozero.reshape(-1,1,3), 3, maxit=None, normalize=False, ATGP_init=True)
print(extract)
mix_pixels = hsv_sample.T
components = extract.T
weights = np.zeros((components.shape[1], mix_pixels.shape[1]))
for i in range(mix_pixels.shape[1]):
    weights[:, i], _ = nnls(components, mix_pixels[:, i])
em1 = weights.T.reshape((m, n, 3))[:, :, 0]
em2 = weights.T.reshape((m, n, 3))[:, :, 1]
em3 = weights.T.reshape((m, n, 3))[:, :, 2]
em_all = np.array([em1,em2,em3])
baocun2(em_all,'D:/xin/无人机覆盖度模型/一种新的SAV提取方法/无人机裁剪后的数据/无人机76-80part3_am(NFINDR3-HSV-no0).tif')

[[ 27. 140. 102.]
 [ 30.  46. 110.]
 [ 40.  77. 249.]]


In [81]:
#NFINDR-RGB（no-0）-3
nfindr = eea.NFINDR()
extract = nfindr.extract(rgb_sample_nozero.reshape(-1,1,3), 3, maxit=None, normalize=False, ATGP_init=True)
print(extract)
mix_pixels = rgb_sample.T
components = extract.T
weights = np.zeros((components.shape[1], mix_pixels.shape[1]))
for i in range(mix_pixels.shape[1]):
    weights[:, i], _ = nnls(components, mix_pixels[:, i])
em1 = weights.T.reshape((m, n, 3))[:, :, 0]
em2 = weights.T.reshape((m, n, 3))[:, :, 1]
em3 = weights.T.reshape((m, n, 3))[:, :, 2]
em_all = np.array([em1,em2,em3])
baocun2(em_all,'D:/xin/无人机覆盖度模型/一种新的SAV提取方法/无人机裁剪后的数据/无人机76-80part3_am(NFINDR3-RGB-no0).tif')

[[159. 100. 188.]
 [ 98.  69.  92.]
 [179. 176. 217.]]


In [31]:
#NFINDR-RGB（0）-4（3）
nfindr = eea.NFINDR()
extract = nfindr.extract(RGB_img, 4, maxit=None, normalize=False, ATGP_init=True)
print(extract)
mix_pixels = rgb_sample.T
components = extract[0:3,:].T
weights = np.zeros((components.shape[1], mix_pixels.shape[1]))
for i in range(mix_pixels.shape[1]):
    weights[:, i], _ = nnls(components, mix_pixels[:, i])
em1 = weights.T.reshape((m, n, 3))[:, :, 0]
em2 = weights.T.reshape((m, n, 3))[:, :, 1]
em3 = weights.T.reshape((m, n, 3))[:, :, 2]
em_all = np.array([em1,em2,em3])
baocun2(em_all,'D:/xin/无人机覆盖度模型/一种新的SAV提取方法/无人机裁剪后的数据/无人机76-80part3_am(NFINDR3-RGB).tif')

[[173. 215. 173.]
 [ 95. 180. 143.]
 [ 93. 136. 145.]
 [  0.   0.   0.]]


In [37]:
#手动纯净像元RGB
shoudong = np.array([[80,144,144]
                    ,[84,154,141]
                    ,[97,175,148]])
mix_pixels = rgb_sample.T
components = shoudong.T
weights = np.zeros((components.shape[1], mix_pixels.shape[1]))
for i in range(mix_pixels.shape[1]):
    weights[:, i], _ = nnls(components, mix_pixels[:, i])
em1 = weights.T.reshape((m, n, 3))[:, :, 0]
em2 = weights.T.reshape((m, n, 3))[:, :, 1]
em3 = weights.T.reshape((m, n, 3))[:, :, 2]
em_all = np.array([em1,em2,em3])
baocun2(em_all,'D:/xin/无人机覆盖度模型/一种新的SAV提取方法/无人机裁剪后的数据/无人机76-80part3_am(shoudong3-RGB).tif')

In [125]:
#手动纯净像元HSV
shoudong = np.array([[29,81,116]
                    ,[35,114,154]
                    ,[40,112,175]])
mix_pixels = hsv_sample.T
components = shoudong.T
weights = np.zeros((components.shape[1], mix_pixels.shape[1]))
for i in range(mix_pixels.shape[1]):
    weights[:, i], _ = nnls(components, mix_pixels[:, i])
em1 = weights.T.reshape((m, n, 3))[:, :, 0]
em2 = weights.T.reshape((m, n, 3))[:, :, 1]
em3 = weights.T.reshape((m, n, 3))[:, :, 2]
em_all = np.array([em1,em2,em3])
baocun2(em_all,'D:/xin/无人机覆盖度模型/一种新的SAV提取方法/无人机裁剪后的数据/无人机76-80part3_am(shoudong3-hsv).tif')

In [20]:
################原始fcls
def _numpy_to_cvxopt_matrix(A):
    from cvxopt import matrix
    A = np.array(A, dtype=np.float64)
    if A.ndim == 1:
        return matrix(A, (A.shape[0], 1), 'd')
    else:
        return matrix(A, A.shape, 'd')


def _numpy_None_vstack(A1, A2):
    if A1 is None:
        return A2
    else:
        return np.vstack([A1, A2])


def _numpy_None_concatenate(A1, A2):
    if A1 is None:
        return A2
    else:
        return np.concatenate([A1, A2])


def _normalize(M):

    minVal = np.min(M)
    maxVal = np.max(M)

    Mn = M - minVal

    if maxVal == minVal:
        return np.zeros(M.shape)
    else:
        return Mn / (maxVal-minVal)


class FCLS(object):
    def __init__(self):
        self.amap = None
        self.m = None
        self.n = None
        self.q = None

    def map(self, M, U, normalize=False, mask=None):

        h, w, numBands = M.shape
        if normalize:
            M = _normalize(M)
            U = _normalize(U)
        Mr = np.reshape(M, (w*h, numBands))
        if isinstance(mask, np.ndarray):
            m = np.reshape(mask, (w*h))
            cMr = _compress(Mr, m)
            c_amap2D = fcls_real(cMr, U)
            amap2D = _expand(c_amap2D, m, w*h, U.shape[0])
        else:
            amap2D = fcls_real(Mr, U)
        self.amap = np.reshape(amap2D, (h, w, U.shape[0]))
        self.m = h
        self.n = w
        self.q = U.shape[0]
        return self.amap

    def __str__(self):
        return 'pysptools.abundance_maps.amaps_int.FCLS object, maps: {0}x{1}x{2}'.format(self.m, self.n, self.q)

    def plot(self, path, mask= None, interpolation='none', colorMap='jet', columns=None, suffix=None):
        _plot_abundance_map(self.amap, path, 'FCLS', mask, interpolation, colorMap, columns, suffix)

    def display(self, mask= None, interpolation='none', colorMap='jet', columns=None, suffix=None):
        _plot_abundance_map(self.amap, None, 'FCLS', mask, interpolation, colorMap, columns, suffix)


def fcls_real(M, U):
    solvers.options['show_progress'] = False
    N, p1 = M.shape
    nvars, p2 = U.shape

    C = _numpy_to_cvxopt_matrix(U.T)
    Q = C.T * C

    lb_A = -np.eye(nvars)
    lb = np.repeat(0, nvars)
    A = _numpy_None_vstack(None, lb_A)
    b = _numpy_None_concatenate(None, -lb)
    A = _numpy_to_cvxopt_matrix(A)
    b = _numpy_to_cvxopt_matrix(b)

    Aeq = _numpy_to_cvxopt_matrix(np.ones((1,nvars)))
    beq = _numpy_to_cvxopt_matrix(np.ones(1))

    M = np.array(M, dtype=np.float64)
    X = np.zeros((N, nvars), dtype=np.float32)
    for n1 in range(N):
        d = matrix(M[n1], (p1, 1), 'd')
        q = - d.T * C
        sol = solvers.qp(Q, q.T, A, b, Aeq, beq, None, None)['x']
        X[n1] = np.array(sol).squeeze()
    return X



In [27]:
#fcls_real(hsv_sample ,np.array([[29,81,116]
#                    ,[35,114,154]
#                    ,[40,112,175]]))

In [27]:
################改进fcls
import numpy as np
from cvxopt import matrix, solvers

def _normalize(M):
    minVal = np.min(M)
    maxVal = np.max(M)
    Mn = M - minVal
    if maxVal == minVal:
        return np.zeros(M.shape)
    else:
        return Mn / (maxVal - minVal)

def fcls_real(M, U):
    solvers.options['show_progress'] = False
    N, p1 = M.shape
    nvars, p2 = U.shape

    C = matrix(U.T, (p2, nvars), 'd')
    Q = C.T * C

    lb_A = -np.eye(nvars)
    lb = np.repeat(0, nvars)
    A = matrix(lb_A, (nvars, nvars), 'd')
    b = matrix(lb, (nvars, 1), 'd')

    Aeq = matrix(np.ones((1, nvars)), (1, nvars), 'd')
    beq = matrix(np.ones(1), (1, 1), 'd')

    X = np.zeros((N, nvars), dtype=np.float32)
    for n1 in range(N):
        d = matrix(M[n1], (p1, 1), 'd')
        q = - d.T * C
        sol = solvers.qp(Q, q.T, A, b, Aeq, beq, None, None)['x']
        X[n1] = np.array(sol).squeeze()

    return X

M = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])  # 数据矩阵 M
U = np.array([[1, 2, 3], [4, 5, 6]])  # 端元成分矩阵 U

# 调用函数进行约束线性最小二乘拟合
result = fcls_real(M, U)
result

array([[9.9961239e-01, 3.8760836e-04],
       [1.0239016e-03, 9.9897611e-01],
       [9.8368467e-08, 9.9999988e-01]], dtype=float32)

In [28]:
################改进fcls2
import numpy as np
from cvxopt import matrix, solvers
from joblib import Parallel, delayed

def fcls_real(M, U):
    solvers.options['show_progress'] = False
    N, p1 = M.shape
    nvars, p2 = U.shape

    C = matrix(U.T, (p2, nvars), 'd')
    Q = C.T * C

    lb_A = -np.eye(nvars)
    lb = np.repeat(0, nvars)
    A = matrix(lb_A, (nvars, nvars), 'd')
    b = matrix(lb, (nvars, 1), 'd')

    Aeq = matrix(np.ones((1, nvars)), (1, nvars), 'd')
    beq = matrix(np.ones(1), (1, 1), 'd')

    def solve_optimization(n1):
        d = matrix(M[n1], (p1, 1), 'd')
        q = - d.T * C
        sol = solvers.qp(Q, q.T, A, b, Aeq, beq, None, None)['x']
        return np.array(sol).squeeze()

    X = Parallel(n_jobs=-1)(delayed(solve_optimization)(n1) for n1 in range(N))
    X = np.vstack(X)

    return X

M = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])  # 数据矩阵 M
U = np.array([[1, 2, 3], [4, 5, 6]])  # 端元成分矩阵 U

result = fcls_real(M, U)
result

array([[9.99612392e-01, 3.87608375e-04],
       [1.02390165e-03, 9.98976098e-01],
       [9.83684645e-08, 9.99999902e-01]])

In [45]:
#高斯混合模型计算权重
import numpy as np

def gaussian(x, mean, std):
    return np.exp(-0.5 * ((x - mean) / std) ** 2) / (std * np.sqrt(2 * np.pi))

def estimate_abundance(features, means, stds, weights, max_iterations=100, tolerance=1e-6):
    num_features = features.shape[1]
    num_endmembers = len(means)

    # 初始化丰度矩阵
    abundances = np.random.rand(features.shape[0], num_endmembers)
    abundances /= np.sum(abundances, axis=1, keepdims=True)

    for _ in range(max_iterations):
        # E步：计算每个像元对应每个地物的后验概率
        posteriors = np.zeros((features.shape[0], num_endmembers))
        for i in range(num_endmembers):
            posteriors[:, i] = weights[i] * np.prod(gaussian(features, means[i], stds[i]), axis=1)
        posteriors /= np.sum(posteriors, axis=1, keepdims=True)

        # M步：更新丰度矩阵和地物权重
        new_abundances = posteriors / np.sum(posteriors, axis=1, keepdims=True)

        # 计算收敛条件
        diff = np.max(np.abs(new_abundances - abundances))
        if diff < tolerance:
            break

        abundances = new_abundances

    return abundances

# 示例数据
features = np.array([[1.2, 2.5, 3.1],
                     [2.0, 1.8, 2.9],
                     [3.5, 3.2, 2.7],
                     [2.8, 3.0, 1.5]])

means = np.array([[1.0, 2.0, 3.0],
                  [2.5, 1.5, 2.8],
                  [3.2, 3.5, 2.0]])

stds = np.array([[0.5, 0.8, 0.7],
                 [0.6, 0.7, 0.9],
                 [0.7, 0.6, 0.8]])

weights = np.array([0.3, 0.3, 0.3])

# 估计丰度值
abundances = estimate_abundance(features, means, stds, weights)

print("Estimated Abundances:")
print(abundances)

Estimated Abundances:
[[9.67165333e-01 3.10809628e-02 1.75370455e-03]
 [2.14167419e-01 7.82798825e-01 3.03375590e-03]
 [2.36277777e-06 2.05893405e-02 9.79408297e-01]
 [1.62630095e-04 5.33385065e-02 9.46498863e-01]]


In [44]:
abundances = estimate_abundance(features, means, stds, weights)

print("Estimated Abundances:")
print(abundances)

Estimated Abundances:
[[8.79035272e-01 1.12995211e-01 7.96951725e-03]
 [6.37302222e-02 9.31755974e-01 4.51380370e-03]
 [4.74510421e-07 1.65396116e-02 9.83459914e-01]
 [3.28810623e-05 4.31365857e-02 9.56830533e-01]]
